In [1]:
from artap.problem import Problem
from artap.results import Results
from artap.surrogate_smt import SurrogateModelSMT
from artap.operators import LHSGenerator
from artap.algorithm_sweep import SweepAlgorithm
# from artap.algorithm_nlopt import NLopt, LN_BOBYQA
from artap.algorithm_genetic import NSGAII
# 
import numpy as np
from smt.surrogate_models import RBF, MGP

In [34]:
class ProblemBranin(Problem):
    def set(self):
        self.parameters = [{'name': 'x_1', 'initial_value': 3, 'bounds': [-10, 10]},
                           {'name': 'x_2', 'initial_value': 10, 'bounds': [-10, 10]}]
        self.costs = [{'name': 'F'}]

    def evaluate(self, individual):
        x = individual.vector
        y = (x[0] + 2 * x[1] - 7) ** 2 + (2 * x[0] + x[1] - 5) ** 2

        return [y]

    def predict(self, individual):
        sigma_MGP, sigma_KRG = self.surrogate.predict_variances(individual.vector, True)

        if sigma_MGP < 1e-3:
            self.surrogate.train_step *= 1.3

            value_problem = self.evaluate(individual)
            value_surrogate = self.surrogate.predict(individual.vector)
            # print(sigma_MGP, sigma_KRG, individual.vector, value_problem, value_surrogate)

            return value_surrogate

        return None

In [35]:
problem = ProblemBranin()

# surrogate
problem.surrogate = SurrogateModelSMT(problem)
# problem.surrogate.add_data(x, y)

# set custom regressor
problem.surrogate.regressor = MGP(theta0=[1e-2], n_comp=2, print_prediction=False)

# DoE - Latin - Hypercube
gen = LHSGenerator(parameters=problem.parameters)
gen.init(number=20)

algorithm_sweep = SweepAlgorithm(problem, generator=gen)
algorithm_sweep.run()

# train model
problem.surrogate.train()

# set train step
problem.surrogate.train_step = 30

# run optimization
algorithm = NSGAII(problem)
algorithm.options['max_population_number'] = 20
algorithm.options['max_population_size'] = 10
algorithm.options['max_processes'] = 1
algorithm.run()

2021-02-03 14:55:47,722 (INFO): ProblemBranin-718588 - run (29) - Sweep: elapsed time: 0.002454042434692383 s
2021-02-03 14:55:47,910 (INFO): ProblemBranin-718588 - run (146) - NSGA_II: 20/200
2021-02-03 14:55:48,236 (INFO): ProblemBranin-718588 - run (173) - NSGA_II: elapsed time: 0.3267245292663574 s


In [36]:
b = Results(problem)
optimum = b.find_optimum('F_1')  # Takes last cost function
print(optimum.vector, optimum.costs)

problem.logger.info("surrogate: predict / eval counter: {0:5.0f} / {1:5.0f}, total: {2:5.0f}".format(
    problem.surrogate.predict_counter,
    problem.surrogate.eval_counter,
    problem.surrogate.predict_counter + problem.surrogate.eval_counter))

2021-02-03 14:55:49,096 (INFO): ProblemBranin-718588 - <module> (5) - surrogate: predict / eval counter:   209 /    21, total:   230


[1.0043174522050466, 2.877006687395613] [0.06683256]
